In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.simplefilter('ignore', DeprecationWarning)

In [2]:
#data = pd.read_csv('https://dl.dropboxusercontent.com/u/5743203/data/titanic/titanic_train.csv')
data = pd.read_csv('../../../data/clean_data.csv')  # result column, 1 means home win, 2 home draw, 3 home lose
data2 = pd.read_csv('../../../data/clean_data_using_getdummy.csv')  # result column, 1 means home win, 2 home draw, 3 home lose

In [3]:
result_column = data['result']
result_column2 = data2['result']

In [4]:
print(np.mean(result_column == 1))
print(np.mean(result_column == 2))
print(np.mean(result_column == 3))

0.468425071961
0.259531222464
0.272043705575


In [5]:
target = result_column.values
target2 = result_column2.values

In [6]:
# data.columns[0:]
numerical_features = data[data.columns[0:-1]]
numerical_features2 = data2[data2.columns[0:-1]]

In [7]:
features_array = numerical_features.values
features_array2 = numerical_features2.values

In [8]:
from sklearn.cross_validation import train_test_split

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)
features_train2, features_test2, target_train2, target_test2 = train_test_split(
    features_array2, target2, test_size=0.20, random_state=0)
print(len(features_train))
print(len(features_train2))
print(len(features_test2))

13618
13618
3405


In [9]:
def is_unexpected(features, target, threshold):
    b365_odds = features[:, 16 : 19]
    return (((np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold) 
            & (np.argmin(b365_odds, axis = 1) + 1 != target))

def correct_predict_based_on_odds(features, target, threshold):
    b365_odds = features[:, 16:19]
    new_features = features[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    new_target = target[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    b365_odds = b365_odds[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    miss_predicts_rate = np.mean(np.argmin(b365_odds, axis = 1) + 1 != new_target)

    return new_features, new_target, miss_predicts_rate

def miss_predict_based_on_odds(features, target):
    b365_odds = features[:, 16 : 19]
    return np.argmin(b365_odds, axis = 1) + 1 != target    

def is_unexpected2(features, target, threshold):
    b365_odds = features[:, 378 : 381]  # This needs to be changed based on different result.csv
    return (((np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold) 
            & (np.argmin(b365_odds, axis = 1) + 1 != target))

def correct_predict_based_on_odds2(features, target, threshold):
    b365_odds = features[:, 378:381]  # This needs to be changed based on different result.csv
    new_features = features[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    new_target = target[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    b365_odds = b365_odds[(np.amax(b365_odds, axis = 1) - np.amin(b365_odds, axis = 1)) > threshold]
    miss_predicts_rate = np.mean(np.argmin(b365_odds, axis = 1) + 1 != new_target)

    return new_features, new_target, miss_predicts_rate

def miss_predict_based_on_odds2(features, target):
    b365_odds = features[:, 378 : 381] # This needs to be changed based on different result.csv
    return np.argmin(b365_odds, axis = 1) + 1 != target  

for threshold in np.arange(0.5, 5, 0.5):
    new_features2, new_target2, miss_predicts_rate2 = correct_predict_based_on_odds2(features_train2, target_train2, threshold)
    print(len(new_features2), miss_predicts_rate2)
    
print(np.mean(miss_predict_based_on_odds2(features_train2, target_train2)))

13187 0.787214681125
10701 0.805625642463
8397 0.827557460998
6984 0.838487972509
5917 0.850430961636
5006 0.860966839792
4217 0.873843964904
3529 0.883536412581
3010 0.896013289037
0.784182699368


In [10]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(features_train)  
X_train = scaler.transform(features_train)  
# apply same transformation to test data
X_test = scaler.transform(features_test)  
# print(X_train[0:5])

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler  


In [12]:
clf = LogisticRegression(C=1, n_jobs = -1)
# clf.fit(X_train, target_train)

# from sklearn.svm import SVC
# clf = SVC(C = 1, kernel = 'rbf', probability = True)
# clf.fit(X_train, target_train)

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(min_samples_split = 10, n_estimators = 1000, criterion='entropy')
# clf.fit(X_train, target_train)

# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier(n_estimators = 1000)
# clf.fit(X_train, target_train)

# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()
# clf.fit(X_train, target_train)

for threshold in np.arange(0.1, 7, 0.5):
    new_features_train, new_target_train, miss_predicts_rate = correct_predict_based_on_odds(features_train, target_train, threshold)
    new_features_test, new_target_test, miss_predicts_rate_test = correct_predict_based_on_odds(features_test, target_test, threshold)
    print('Odds Difference Threshold to remove sample data: ', threshold)
    print('Number of training data under this odds difference threshold: ', len(new_features_train))
    print('Number of test data under this odds difference threshold: ', len(new_features_test))
    
    # feature scaling
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(new_features_train)  
    X_train = scaler.transform(new_features_train)  
    # apply same transformation to test data
    X_test = scaler.transform(new_features_test)  
    
    
    clf.fit(X_train, new_target_train)
    target_predicted_proba = clf.predict_proba(X_test)
    # print(target_predicted_proba)
    target_predicted = clf.predict(X_test)
    prob_threshold = 0.75
    
    print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
          ' : ', np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    print('Accuracy with this prob_threshold: ', np.mean((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)) 
          / np.mean(np.amax(target_predicted_proba, axis = 1) > prob_threshold))
    #print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    print('Total accuracy: ', accuracy_score(new_target_test, target_predicted))
    X_test_greater_than_prob_threshold = features_train[np.amax(target_predicted_proba, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
    print('\n\n')

Odds Difference Threshold to remove sample data:  0.1
Number of training data under this odds difference threshold:  13616
Number of test data under this odds difference threshold:  3405
Percentage of predictions with the Prob_threshold of  0.75  :  0.0825256975037
Accuracy with this prob_threshold:  0.779359430605
Total accuracy:  0.51718061674



Odds Difference Threshold to remove sample data:  0.6
Number of training data under this odds difference threshold:  13078
Number of test data under this odds difference threshold:  3252
Percentage of predictions with the Prob_threshold of  0.75  :  0.0870233702337
Accuracy with this prob_threshold:  0.780918727915
Total accuracy:  0.524292742927



Odds Difference Threshold to remove sample data:  1.1
Number of training data under this odds difference threshold:  10007
Number of test data under this odds difference threshold:  2477
Percentage of predictions with the Prob_threshold of  0.75  :  0.111021396851
Accuracy with this prob_threshol

In [ ]:
clf = LogisticRegression(C=1, n_jobs = -1)


# from sklearn.svm import SVC
# clf = SVC(C = 1, kernel = 'poly', probability = True)


# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(min_samples_split = 10, n_estimators = 1000, criterion='entropy')


# from sklearn.ensemble import AdaBoostClassifier
# clf = AdaBoostClassifier(n_estimators = 1000)


# from sklearn.naive_bayes import GaussianNB
# clf = GaussianNB()


for threshold in np.arange(0.1, 7, 0.5):
    new_features_train2, new_target_train2, miss_predicts_rate2 = correct_predict_based_on_odds2(features_train2, target_train2, threshold)
    new_features_test2, new_target_test2, miss_predicts_rate_test2 = correct_predict_based_on_odds2(features_test2, target_test2, threshold)
    print('Odds Difference Threshold to remove sample data: ', threshold)
    print('Number of training data under this odds difference threshold: ', len(new_features_train2))
    print('Number of test data under this odds difference threshold: ', len(new_features_test2))
    
    # feature scaling
    scaler = StandardScaler()  
    # Don't cheat - fit only on training data
    scaler.fit(new_features_train2)  
    X_train2 = scaler.transform(new_features_train2)  
    # apply same transformation to test data
    X_test2 = scaler.transform(new_features_test2)  
    
    
    clf.fit(X_train2, new_target_train2)
    target_predicted_proba2 = clf.predict_proba(X_test2)
    # print(target_predicted_proba)
    target_predicted2 = clf.predict(X_test2)
    prob_threshold = 0.70
    
    print('Percentage of predictions with the Prob_threshold of ', prob_threshold, 
          ' : ', np.mean(np.amax(target_predicted_proba2, axis = 1) > prob_threshold))
    print('Accuracy with this prob_threshold: ', np.mean((target_predicted2 == new_target_test2) & (np.amax(target_predicted_proba2, axis = 1) > prob_threshold)) 
          / np.mean(np.amax(target_predicted_proba2, axis = 1) > prob_threshold))
    #print(np.mean((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted == new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    #print(np.sum((target_predicted != new_target_test) & (np.amax(target_predicted_proba, axis = 1) > prob_threshold)))
    print('Total accuracy: ', accuracy_score(new_target_test2, target_predicted2))
    X_test_greater_than_prob_threshold2 = features_train[np.amax(target_predicted_proba2, axis = 1) > prob_threshold, ]
#     print(X_test_greater_than_prob_threshold[0 : 2, ])
    print('\n\n')

Odds Difference Threshold to remove sample data:  0.1
Number of training data under this odds difference threshold:  13614
Number of test data under this odds difference threshold:  3404
Percentage of predictions with the Prob_threshold of  0.7  :  0.320505287897
Accuracy with this prob_threshold:  0.925756186984
Total accuracy:  0.647179788484



Odds Difference Threshold to remove sample data:  0.6
Number of training data under this odds difference threshold:  13108
Number of test data under this odds difference threshold:  3267
Percentage of predictions with the Prob_threshold of  0.7  :  0.341597796143
Accuracy with this prob_threshold:  0.914874551971
Total accuracy:  0.660238751148



Odds Difference Threshold to remove sample data:  1.1
Number of training data under this odds difference threshold:  10094
Number of test data under this odds difference threshold:  2508
Percentage of predictions with the Prob_threshold of  0.7  :  0.552631578947
Accuracy with this prob_threshold:  